In [73]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## MSE

![title](LossFunction/MSE.jpg)

In [83]:
prediction = tf.constant([[1.0, 4.0, 2.0], [1.0, 2.0, 3.0], [2.0, 3.0, 1.0]])
ground_truth = tf.constant([[1.0, 1.0, 1.0], [2.0, 100.0, 2.0], [1.0, 1.0, 1.0]])

## 第一種方法算

In [75]:
#Square_Error = tf.square(ground_truth -prediction)
Square_Error = tf.square(ground_truth -prediction)
mse = tf.reduce_mean(Square_Error) 
with tf.Session() as sess:
    #print(sess.run(Square_Error))
    print(sess.run(mse))

1.8888888


## 第二種方法

In [76]:
#  sum(error ** 2) / 2
mse1 =(2.0*tf.nn.l2_loss(ground_truth - prediction))/tf.reshape(prediction,[-1]).get_shape().as_list()[0]
with tf.Session() as sess:
    #print(tf.reshape(prediction,[-1]).get_shape().as_list()[0])
    print(sess.run(mse1))

1.8888888


## 第三種方法

In [77]:
mse2 = tf.losses.mean_squared_error(ground_truth, prediction)
with tf.Session() as sess:
    print(sess.run(mse2)) #1.8888888

1.8888888


## MAE

![title](LossFunction/MAE.jpg)

In [78]:
##按照公式
residuals = tf.abs(tf.subtract(ground_truth ,prediction))
mae=tf.reduce_mean(residuals)
with tf.Session() as sess:
    print(sess.run(mae))
    
##套用API
mae1 = tf.losses.absolute_difference(ground_truth, prediction)
with tf.Session() as sess:
    print(sess.run(mae))

1.0
1.0


## Huber loss

![title](LossFunction/HuberLoss.png)

In [79]:
## 參考：https://blog.csdn.net/hyk_1996/article/details/79570915
## 按照原理
def huber_loss(labels, predictions, delta=1.0):
    error = tf.abs(tf.subtract(ground_truth ,prediction))
    condition = tf.less(error, delta)
    small_res = 0.5 * tf.square(error) ##誤差小遵照L2
    large_res = delta * error - 0.5 * tf.square(delta) ## 誤差大採用線性誤差
    return tf.where(condition, small_res, large_res)

Huber=tf.reduce_mean(huber_loss(ground_truth, prediction, delta=2.0))
with tf.Session() as sess:
    print(sess.run(Huber))

## Call Tensorflow API
Huber_loss=tf.losses.huber_loss(ground_truth,prediction,delta=2.0)
with tf.Session() as sess:
    print(sess.run(Huber_loss))

0.8888889
0.8888889


In [87]:
loss_1=tf.reduce_mean(huber_loss(ground_truth, prediction, delta=1.0))
loss_2=tf.reduce_mean(huber_loss(ground_truth, prediction, delta=2.0))
loss_3=tf.reduce_mean(huber_loss(ground_truth, prediction, delta=3.0))
loss_5=tf.reduce_mean(huber_loss(ground_truth, prediction, delta=5.0))
loss_10=tf.reduce_mean(huber_loss(ground_truth, prediction, delta=10.0))

In [88]:
with tf.Session() as sess:
    print(sess.run(loss_1))
    print(sess.run(loss_2))
    print(sess.run(loss_3))
    print(sess.run(loss_5))
    print(sess.run(loss_10))

11.5
22.444445
33.11111
54.0
104.27778


## CE

![title](LossFunction/CrossEntropy.png)

In [89]:
prediction = tf.constant([[1.0, 4.0, 2.0], [1.0, 2.0, 3.0], [2.0, 3.0, 1.0]])
ground_truth = tf.constant([[0, 1, 0], [0, 0, 1], [1, 0, 0]],dtype=tf.float32) ## OneHotLabel
ground_truth_1= tf.constant([1,2,0]) ## index label

In [91]:


##按照原理
log_sm_vals = tf.nn.log_softmax(prediction)
individual_loss = tf.reduce_sum(-1.0*tf.math.multiply(ground_truth, log_sm_vals), axis=1)
loss = tf.reduce_mean(individual_loss)
with tf.Session() as sess:
    #print(sess.run(individual_loss))
    print(sess.run(loss)) ##0.66168594
    
## API-1
cross_entropy_sum=tf.nn.softmax_cross_entropy_with_logits_v2(ground_truth,prediction)
cross_entropy=tf.reduce_mean(cross_entropy_sum)
with tf.Session() as sess:
    print(sess.run(cross_entropy)) ##0.66168594
    
## API-2 Sparse_Softmax，輸入不是OneHot
sparse=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=ground_truth_1,logits=prediction))
with tf.Session() as sess:
    print(sess.run(sparse)) ##0.66168594
    
    
##與MSE的差異
mse = tf.losses.mean_squared_error(ground_truth, tf.nn.softmax(prediction))
with tf.Session() as sess:
    print(sess.run(mse)) ##0.13781698

0.66168594
0.66168594
0.66168594
0.13781698


In [92]:
## Binary Cross Entropy
sigmoid=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=[0.,1.,0.],logits=[0.23,1.23,1.72]))

with tf.Session() as sess:
    print(sess.run(sigmoid)) ##0.98529524

0.98529524


## MSE v.s CE

![title](LossFunction/CrossEntropyMSE.gif)

In [107]:
prediction = 0.98

In [108]:
print(-1*np.log(prediction))
print(np.mean([(1-prediction)**2,((1-prediction)-0)**2]))

0.020202707317519466
0.0004000000000000007


## Focal Loss

![title](LossFunction/FocalLossFormula.png)

In [109]:
class FocalLoss():
    def __init__(self, gamma=2, alpha=[0.1,0.2,0.4], size_average=True):
        self.gamma = gamma
        self.alpha = alpha
        self.size_average = size_average

    def forward(self, prediction, ground_truth):

        softmax_prediction=tf.nn.softmax(prediction)
        softmax_prediction=tf.reduce_sum(softmax_prediction*ground_truth,1)
        logpt=tf.log(softmax_prediction)

        if self.alpha is not None:
            at = tf.constant(self.alpha)

        loss = -1*at * (1-softmax_prediction)**self.gamma * logpt
        if self.size_average: return tf.reduce_mean(loss)
        else: return tf.reduce_sum(loss)
        

In [111]:
prediction = tf.constant([[1.0, 4.0, 2.0], [1.0, 2.0, 3.0], [2.0, 3.0, 1.0]],dtype=tf.float32)
ground_truth = tf.constant([[0, 1, 0], [0, 0, 1], [1, 0, 0]],dtype=tf.float32) ## OneHotLabel

## Alpha為頻率倒數，gamma先設為2
with tf.Session() as sess:
    Focal=FocalLoss( )
    print(sess.run(Focal.forward(prediction,ground_truth))) #0.11024303
    

##當gamma=0,alpha=1時，就是CE
with tf.Session() as sess:
    Focal=FocalLoss(gamma=0,alpha=[1.,1.,1.])
    print(sess.run(Focal.forward(prediction,ground_truth))) #0.661686
    

##會與上方得到數值相同
cross_entropy_sum=tf.nn.softmax_cross_entropy_with_logits_v2(ground_truth,prediction)
cross_entropy=tf.reduce_mean(cross_entropy_sum)
with tf.Session() as sess:
    print(sess.run(cross_entropy)) #0.661686

0.11024303
0.661686
0.66168594


## Center Loss

![title](LossFunction/CenterLossformula.png)

In [113]:
import tensorflow as tf
##參考 https://github.com/EncodeTS/TensorFlow_Center_Loss
def get_center_loss(features, labels, alpha=0.1, num_classes=3):

    ##Embedding的維度
    len_features = features.get_shape()[1]

    ##初始化center
    centers = tf.get_variable('centers', [num_classes, len_features], dtype=tf.float32,
        initializer=tf.constant_initializer(0), trainable=False)

    
    centers_batch = tf.gather(centers, labels)
    
    ##計算Loss
    loss = tf.nn.l2_loss(features - centers_batch)
    
    
    #更新各個類別的Center
    diff = centers_batch - features
    
    unique_label, unique_idx, unique_count = tf.unique_with_counts(labels)
    appear_times = tf.gather(unique_count, unique_idx)
    appear_times = tf.reshape(appear_times, [-1, 1])
    
    diff = diff / tf.cast((1 + appear_times), tf.float32)
    diff = alpha * diff
    
    centers_update_op = tf.scatter_sub(centers, labels, diff)
    
    return loss, centers, centers_update_op

tf.reset_default_graph()
Embedding=tf.Variable([[1,2,],[2,3],[1,2]],dtype=tf.float32)
loss,_,_=get_center_loss(Embedding, [2,0,1], alpha=0.1, num_classes=3)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss))

11.5


## Triplet Loss

![title](LossFunction/FocalLossFormula.png)

In [114]:
tf.reset_default_graph()
##Label
label=tf.constant([0,1,2,1,2,1,0,1],dtype=tf.int32)
##Embedding output
Embedding_out_out=tf.Variable(np.random.randn(8,256),dtype=tf.float32)
##經過L2-normalization
prediction_semi=tf.nn.l2_normalize(Embedding_out_out,axis=1)
##計算 triplet loss
loss_semi=tf.contrib.losses.metric_learning.triplet_semihard_loss(label,prediction_semi)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss_semi))

0.9789365


## Hand On

# Regression Model
## 1. Huber Loss
# Classification Model
## 1.Triplet Loss+Softmax
## 2.Center Loss

## 補充

In [481]:
def Reorganize_data(x, y,class_=13925):
    assert x.shape[0] == y.shape[0]
    
    dataset = {i: [] for i in range(class_)}
    
    for i in range(x.shape[0]):
        dataset[y[i][0]].append(x[i][0])
        
    return dataset

def get_batch(dataset,k=4,class_=13925):
    # Sample BATCH_K random images from each category of the MNIST dataset,
    # returning the data along with its labels
    batch = []
    labels = []
    
    for l in range(class_):
        indices = rm.sample(range(len(dataset[l])), k)
        indices = np.array(indices)

        batch.append([dataset[l][i] for i in indices])
        labels += [l] * k

    batch = np.array(batch).reshape(class_ * k)
    labels = np.array(labels)
    
    
    return batch, labels

def Train_Read(batch_,labels_,batch_batch,batch=100,k=4):
    data_batch_x=[]
    data_batch_y=[]
    global random_choice
    random_choice=rm.sample(batch_batch,int(batch/k))
    for a in random_choice:
        data_batch_x.append(batch_[a:a+k])
        data_batch_y.append(labels_[a:a+k])
    data_batch_x=np.array(data_batch_x).reshape(batch)
    data_batch_y=np.array(data_batch_y).reshape(batch,1)
    Batch_Read=[]
    for path in data_batch_x:
        img = image.load_img(path, target_size=(28, 28))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        x=x.reshape(28, 28, 3)
        Batch_Read.append(x)
    Batch_Read=np.array(Batch_Read)
    return Batch_Read,data_batch_y

In [482]:
#Label_X 是list train_data
#Label_Y 是list train)_y

In [ ]:
dataset_=Reorganize_data(Label_X,Label_Y)

iteration = int((4*13925)/100)
batch_batch=[x for x in range(0,len(dataset_),4)]
batch_,labels_=get_batch(dataset_, 4,class_=13925)


train_batch_x,train_batch_y=Train_Read(batch_,labels_,batch_batch,100,4)
for _number_ in random_choice:
    batch_batch.remove(_number_)